In [94]:
from common import *

In [95]:
db = load_db(download_tar("http://vision.lems.brown.edu/sites/default/files/216db.tar.gz"))

216db exists. Skipping


In [105]:
def test_queries(descriptor, distance, top_n=5):

    desc = [descriptor(im) for im in db.image]
    db["desc"]=desc

    queries = []
    results=[]
    for clas in db.clas.unique():
        row = db.query(f"clas=='{clas}'").iloc[0]

        C = np.stack(db.desc.to_numpy())

        desc = row.desc
        dists = distance( row.desc, C)

        queries.append(pd.DataFrame(data=[row])[["clas","image"]])
        results.append(db.iloc[np.argsort(dists)].head(n=top_n)[["clas","image"]])
    
    return queries, results

In [106]:
def show_query_results(queries, results):
    qDf = {}
    rDf = pd.DataFrame()
    for i, (q, r) in enumerate(zip(queries, results)):
        qDf[f"{q.clas.iloc[0]}"] = q.image.iloc[0]
        rDf[f"{q.clas.iloc[0]}"] = r.image.reset_index(drop=True)

    print("queries:")
    displayImages(pd.DataFrame(data=[qDf]))
    print("results:")
    displayImages(rDf)

In [107]:
def hu_moments(im):
    im = 255*(im==0).astype(np.uint8)
    h=cv2.moments(im)

    return cv2.HuMoments(h).flatten()

def hu_moments_log(im):
    hu = hu_moments(im)
    return -1* np.copysign(1.0, hu) * np.log10(abs(hu))


def l1Dist(vec, dataset):
    return np.linalg.norm( vec.reshape(1,-1) - dataset , axis=-1, ord=1)

In [110]:
queries, results = test_queries(hu_moments, l1Dist, top_n=10)
show_query_results(queries, results)

queries:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABzCAAAAACOKGI6AAADdklEQVRoBb3BCWLiSADAQOn/j9aCL4xtCFl6UmX8OePPGX/O+BXja8ZvCMSXjE/JIr5jfEL24ivGB+RJfMX4mRzEN4wfyYX434yfyAvx/xg/kDcyfs14T34Qv2W8Jz+K3zHekw/EbxhvyUfiF4y35EPxMeMd+Vx8yHhHfiE+Y7wjvxCfMd6QX4mPGG/I74RhvGe8If9TvGG8Jv9fvGS8Jt+IF4xX5FtxyXhBBogLxjUZIs6MazJGnBiXZJQ4Mi7JMHFgXJGB4plxRUaKJ8YFGSx2jAsyWOwYZzJcPBhnMl5sjBP5F2JlHMk/ESvjSP6NWBgH8o/Ewngm/0zMjCfyD8XE2JMXku/FxNiRV0K+FhNjIy+FDBB3xkIWIUchA8SdMZNFIEfJCHFnTGQSN3IUMkTcGCCTmMlRyBBxY4DcxZ1cSAYJMEAgJnIhZJAAA4yVnIWMEmCcyF7cyCABxpHsxY0MExhH8hATGSXAOJJNzGSUAONAVrGQcQLjQFaxkGECjANZxUwGCowDWcRCxgkwnskqFjJQYDyTRaxknADjmSxiJeMEGE9kFSsZJm6MZzKLlYwTN8YTWcVChok744nM4kEGiYnxRGaxkVFiYuzJLB5klJgYezKLBxklJsaezGJHBomJsSOz2JFRYmLsyCz25HsJxMTYkVnsyUvJR0IgJsaOTOJB3ogb+VHITUyMB5nFg7wWM3kvQCAmxkZm8SCvxI7JS3EjEBNjJYvYyAtxIC/FjUBMjJXMYiPX4kReihu5iTvjTpJZPMilOJNXYiIQd8aNPMSDXIkzeSlmQtwZN7KJB7kSZ/JSLIS4M0A2sScncUVeioUQdwbIKp7IUVySV2IlxJ1xI4t4IkdxRV6LhRB3xkTu4kBINnFJXouVEHfGQogjIXmIC/JGrIS4M96RkE1ckSfJTmwk7owfyCauyLNANrGRuDN+IKs4M3kWIKvYCHFnvCerOJOjuJFVPEjcGW/JIs7kJO5kFWfGO7KKC3IQE1nEBeMdWcQFOYqJLOKC8ZbM4oo8i5ms4sz4iUBck71YyCrOjK/IQ6xkFWfGd2QVK9nEmfEtmcVKNnFmfEsmsZFNnBnfk5vYyCbOjAGEeJBNnBnDySYuGKPJQ1wwBpO9ODMGk704MwaTvTgzxpK9uGCMJpu4Yowns7hk/Dnjzxl/zvhz/wHX8lUQ8fABEgAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABaCAAAAACuizbEAAACF0lEQVRoBb3BC3biMADAQOn+h9byKXRLm2DHecwYH2d8nPFxxmnMGGCcQf4Tf5CLuDLWyS/xZMhN3BiL5G9xJU/xxVgio+LBOEzGxTfjGBkXPxhHyLh4YcyTCfHKmCbj4jdjlsyIO4G4MebInLiSu7gwpsikAHmIC2OGzEr+F2CMk3WBMUzWBRiD5AwBxiA5RWCMkVMEGEPkFHFhDJEzxJUxQk4RV8YIOUPcGAPkFHFjvCfniBvjPTlH3BhvyYLkJow74x05LpCbeDLekOMChIR4MvbJcXEhkPHNeEMOiwshJJ6MfXJYXEn8ZOySw+JO4gdjlxwVD8YPxh45KjYZe+SY2GHskUNij7FHjohdxg45IvYZO+SAeMPYJgfEO8Y2mZNkvGNskylhDDC2yYzAGGBskikxyNgkM2KUsUVmxDBjk4yLccYWGRcTjC0yLGYYW2RUTDG2yKCYY2yTETHJ2GTIWzHL+IMkY2Ka8UomxDzjB5kRRxhPMiuOMO5kWhxjXMm8OMi4kmlxlAEyLY4zQKbFcQbIpFhhgMyJJQbIjuSXWGEg2wLkRSwxQP4UX4RAnmKFcWHIi3ghT7HCeDB5iN/kKVYY/zPJ+Is8xQpjlDzFCmOUfIsFxih5ihXGKHmIJcYoeYglxiD5EouMQfIlFhlj5CaMRcYQuYt1xhC5CIllxhBJwlhnjDGJUxhjjLMYH2d8nPFx/wCZfNkBhcdNWAAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA3CAAAAAAa4SpAAAABWElEQVRYCb3BAWKiQADAwOT/j86xaqnaa0WBnTGmM6YzpjOOYGZmZmZmZsYz4yPygbgw3iSfi8HYTPaKwdhGDhCDsYkcIQZjEwNkpxiMN8g+MRjbyV6xMLaTnWIw3iD7xGC8QfaJwdjKkH1iMH4QMjMzMzPZLwbjBzlLDMYzOU0MxhM5TwzGEzlPDMZKzhaDsZKzxWCs5GwxGCs5WwzGlcwQC2OQOWJhLGSSWBgLmSMGA2SWWBjILDEYYDJHLIxB5oiFsZBZAoyFzBJggEwTYIBME2AsZJYAA2SewACZJRYGyDbJL5LX4soA2SIuzMwMYyX3AuRR3BgLeSn+JndikAdxYyzklXhJVjHIg7gxBnkhXpKbuJE78cW4kDvJs9jCJFbyLVbGhXwLeRKfkZv4ZnyRqwB5FB+SIe4ZK0niQu7EoYz/k1Ucy/iFXMXRjD+YcThjOmM6Y7p/c3mdARXOc4QAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA4CAAAAAAJa4PsAAABO0lEQVRYCb3BAXLiMADAQOn/j9aFUNrCAQlj17vGesZ6xnrGesZ6xguSfIuJjKdMHsQsxhPyTExiPJCXYg7jnrwRUxh35K2YwfhFDsQMxg85EjMY3+RYTGDcyBkxzvgip8Q440rOiXHGlZwT44ydnBTjjJ2cFcOMnXwgxhg7+URgci8w45hxIZ8JeS0zHhg3xkb+SuwEMq6MjawRO2Mji8SFcSGLxMbYyDIBxoWsEmBsZJXYGBeyQsaFcSF/IgOBhLgxNjJdvGZcyDRxyNjJpxISMskkiTOMnZwVQowwdnJSjDOu5JwYZ1zJOTHOuJJzYpzxRU6JccYXOSXGGTdyRowzvsmxmMD4IYdiAuMXORAzGL/JezGDcUfeiSmMB/JSzGH8T56KSYxnTB7FLMZzci/mMdYz1jPWM9b7B9JugAEuFMLuAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAIIAAACCCAAAAACvaE8hAAAC80lEQVR4AcXBAY7bOADAQPL/j+a1V2DjbGRHcmxkxvg249uMbzO+zRiTf+JuxpA8xK2MAXkSdzJeyS9xI+OFvIj7GL/JQNzGeCZDcRvjieyIuxgbsivuYmzIvriJ8UOOxE2MH3IkbmI8SJjJSNzDGJCBuIcxJK/iFsaQvIpbGGPyIm5h7JDf4hbGDnkRdzB2yIu4g7FDXsQdjB3yIu5g7JAXcQdjh7yIOxg75FXcwBiTgbiBMSYDcQNjTEbiesaYDMQNjDEZiesZQzIU1zPGZCguZ4zJUFzOGJIdcTVjSHbE1Ywh2RFXM4ZkT1zMGJFdcTFjRPbFtYwR2RfXMkbkQFzKGJEDcSljRI7ElYwBORRXMkbkUFzIGJBjcSFjRI7FdYwBeScuYwzIW3EVY0TeiosYAzIhrmEMyIy4hDEgM+ISxoBMiSsYIzIlLmAMyKT4nDEgs+JjxojMik8ZAzItPmWMyLT4kDEgC+IzxogsiI8YI7IiPmGMy

results:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABzCAAAAACOKGI6AAADdklEQVRoBb3BCWLiSADAQOn/j9aCL4xtCFl6UmX8OePPGX/O+BXja8ZvCMSXjE/JIr5jfEL24ivGB+RJfMX4mRzEN4wfyYX434yfyAvx/xg/kDcyfs14T34Qv2W8Jz+K3zHekw/EbxhvyUfiF4y35EPxMeMd+Vx8yHhHfiE+Y7wjvxCfMd6QX4mPGG/I74RhvGe8If9TvGG8Jv9fvGS8Jt+IF4xX5FtxyXhBBogLxjUZIs6MazJGnBiXZJQ4Mi7JMHFgXJGB4plxRUaKJ8YFGSx2jAsyWOwYZzJcPBhnMl5sjBP5F2JlHMk/ESvjSP6NWBgH8o/Ewngm/0zMjCfyD8XE2JMXku/FxNiRV0K+FhNjIy+FDBB3xkIWIUchA8SdMZNFIEfJCHFnTGQSN3IUMkTcGCCTmMlRyBBxY4DcxZ1cSAYJMEAgJnIhZJAAA4yVnIWMEmCcyF7cyCABxpHsxY0MExhH8hATGSXAOJJNzGSUAONAVrGQcQLjQFaxkGECjANZxUwGCowDWcRCxgkwnskqFjJQYDyTRaxknADjmSxiJeMEGE9kFSsZJm6MZzKLlYwTN8YTWcVChok744nM4kEGiYnxRGaxkVFiYuzJLB5klJgYezKLBxklJsaezGJHBomJsSOz2JFRYmLsyCz25HsJxMTYkVnsyUvJR0IgJsaOTOJB3ogb+VHITUyMB5nFg7wWM3kvQCAmxkZm8SCvxI7JS3EjEBNjJYvYyAtxIC/FjUBMjJXMYiPX4kReihu5iTvjTpJZPMilOJNXYiIQd8aNPMSDXIkzeSlmQtwZN7KJB7kSZ/JSLIS4M0A2sScncUVeioUQdwbIKp7IUVySV2IlxJ1xI4t4IkdxRV6LhRB3xkTu4kBINnFJXouVEHfGQogjIXmIC/JGrIS4M96RkE1ckSfJTmwk7owfyCauyLNANrGRuDN+IKs4M3kWIKvYCHFnvCerOJOjuJFVPEjcGW/JIs7kJO5kFWfGO7KKC3IQE1nEBeMdWcQFOYqJLOKC8ZbM4oo8i5ms4sz4iUBck71YyCrOjK/IQ6xkFWfGd2QVK9nEmfEtmcVKNnFmfEsmsZFNnBnfk5vYyCbOjAGEeJBNnBnDySYuGKPJQ1wwBpO9ODMGk704MwaTvTgzxpK9uGCMJpu4Yowns7hk/Dnjzxl/zvhz/wHX8lUQ8fABEgAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABaCAAAAACuizbEAAACF0lEQVRoBb3BC3biMADAQOn+h9byKXRLm2DHecwYH2d8nPFxxmnMGGCcQf4Tf5CLuDLWyS/xZMhN3BiL5G9xJU/xxVgio+LBOEzGxTfjGBkXPxhHyLh4YcyTCfHKmCbj4jdjlsyIO4G4MebInLiSu7gwpsikAHmIC2OGzEr+F2CMk3WBMUzWBRiD5AwBxiA5RWCMkVMEGEPkFHFhDJEzxJUxQk4RV8YIOUPcGAPkFHFjvCfniBvjPTlH3BhvyYLkJow74x05LpCbeDLekOMChIR4MvbJcXEhkPHNeEMOiwshJJ6MfXJYXEn8ZOySw+JO4gdjlxwVD8YPxh45KjYZe+SY2GHskUNij7FHjohdxg45IvYZO+SAeMPYJgfEO8Y2mZNkvGNskylhDDC2yYzAGGBskikxyNgkM2KUsUVmxDBjk4yLccYWGRcTjC0yLGYYW2RUTDG2yKCYY2yTETHJ2GTIWzHL+IMkY2Ka8UomxDzjB5kRRxhPMiuOMO5kWhxjXMm8OMi4kmlxlAEyLY4zQKbFcQbIpFhhgMyJJQbIjuSXWGEg2wLkRSwxQP4UX4RAnmKFcWHIi3ghT7HCeDB5iN/kKVYY/zPJ+Is8xQpjlDzFCmOUfIsFxih5ihXGKHmIJcYoeYglxiD5EouMQfIlFhlj5CaMRcYQuYt1xhC5CIllxhBJwlhnjDGJUxhjjLMYH2d8nPFx/wCZfNkBhcdNWAAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA3CAAAAAAa4SpAAAABWElEQVRYCb3BAWKiQADAwOT/j86xaqnaa0WBnTGmM6YzpjOOYGZmZmZmZsYz4yPygbgw3iSfi8HYTPaKwdhGDhCDsYkcIQZjEwNkpxiMN8g+MRjbyV6xMLaTnWIw3iD7xGC8QfaJwdjKkH1iMH4QMjMzMzPZLwbjBzlLDMYzOU0MxhM5TwzGEzlPDMZKzhaDsZKzxWCs5GwxGCs5WwzGlcwQC2OQOWJhLGSSWBgLmSMGA2SWWBjILDEYYDJHLIxB5oiFsZBZAoyFzBJggEwTYIBME2AsZJYAA2SewACZJRYGyDbJL5LX4soA2SIuzMwMYyX3AuRR3BgLeSn+JndikAdxYyzklXhJVjHIg7gxBnkhXpKbuJE78cW4kDvJs9jCJFbyLVbGhXwLeRKfkZv4ZnyRqwB5FB+SIe4ZK0niQu7EoYz/k1Ucy/iFXMXRjD+YcThjOmM6Y7p/c3mdARXOc4QAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA4CAAAAAAJa4PsAAABO0lEQVRYCb3BAXLiMADAQOn/j9aFUNrCAQlj17vGesZ6xnrGesZ6xguSfIuJjKdMHsQsxhPyTExiPJCXYg7jnrwRUxh35K2YwfhFDsQMxg85EjMY3+RYTGDcyBkxzvgip8Q440rOiXHGlZwT44ydnBTjjJ2cFcOMnXwgxhg7+URgci8w45hxIZ8JeS0zHhg3xkb+SuwEMq6MjawRO2Mji8SFcSGLxMbYyDIBxoWsEmBsZJXYGBeyQsaFcSF/IgOBhLgxNjJdvGZcyDRxyNjJpxISMskkiTOMnZwVQowwdnJSjDOu5JwYZ1zJOTHOuJJzYpzxRU6JccYXOSXGGTdyRowzvsmxmMD4IYdiAuMXORAzGL/JezGDcUfeiSmMB/JSzGH8T56KSYxnTB7FLMZzci/mMdYz1jPWM9b7B9JugAEuFMLuAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAIIAAACCCAAAAACvaE8hAAAC80lEQVR4AcXBAY7bOADAQPL/j+a1V2DjbGRHcmxkxvg249uMbzO+zRiTf+JuxpA8xK2MAXkSdzJeyS9xI+OFvIj7GL/JQNzGeCZDcRvjieyIuxgbsivuYmzIvriJ8UOOxE2MH3IkbmI8SJjJSNzDGJCBuIcxJK/iFsaQvIpbGGPyIm5h7JDf4hbGDnkRdzB2yIu4g7FDXsQdjB3yIu5g7JAXcQdjh7yIOxg75FXcwBiTgbiBMSYDcQNjTEbiesaYDMQNjDEZiesZQzIU1zPGZCguZ4zJUFzOGJIdcTVjSHbE1Ywh2RFXM4ZkT1zMGJFdcTFjRPbFtYwR2RfXMkbkQFzKGJEDcSljRI7ElYwBORRXMkbkUFzIGJBjcSFjRI7FdYwBeScuYwzIW3EVY0TeiosYAzIhrmEMyIy4hDEgM+ISxoBMiSsYIzIlLmAMyKT4nDEgs+JjxojMik8ZAzItPmWMyLT4kDEgC+IzxogsiI8YI7IiPmGMy

In [104]:
queries, results = test_queries(hu_moments_log, l1Dist)
show_query_results(queries, results)

queries:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABzCAAAAACOKGI6AAADdklEQVRoBb3BCWLiSADAQOn/j9aCL4xtCFl6UmX8OePPGX/O+BXja8ZvCMSXjE/JIr5jfEL24ivGB+RJfMX4mRzEN4wfyYX434yfyAvx/xg/kDcyfs14T34Qv2W8Jz+K3zHekw/EbxhvyUfiF4y35EPxMeMd+Vx8yHhHfiE+Y7wjvxCfMd6QX4mPGG/I74RhvGe8If9TvGG8Jv9fvGS8Jt+IF4xX5FtxyXhBBogLxjUZIs6MazJGnBiXZJQ4Mi7JMHFgXJGB4plxRUaKJ8YFGSx2jAsyWOwYZzJcPBhnMl5sjBP5F2JlHMk/ESvjSP6NWBgH8o/Ewngm/0zMjCfyD8XE2JMXku/FxNiRV0K+FhNjIy+FDBB3xkIWIUchA8SdMZNFIEfJCHFnTGQSN3IUMkTcGCCTmMlRyBBxY4DcxZ1cSAYJMEAgJnIhZJAAA4yVnIWMEmCcyF7cyCABxpHsxY0MExhH8hATGSXAOJJNzGSUAONAVrGQcQLjQFaxkGECjANZxUwGCowDWcRCxgkwnskqFjJQYDyTRaxknADjmSxiJeMEGE9kFSsZJm6MZzKLlYwTN8YTWcVChok744nM4kEGiYnxRGaxkVFiYuzJLB5klJgYezKLBxklJsaezGJHBomJsSOz2JFRYmLsyCz25HsJxMTYkVnsyUvJR0IgJsaOTOJB3ogb+VHITUyMB5nFg7wWM3kvQCAmxkZm8SCvxI7JS3EjEBNjJYvYyAtxIC/FjUBMjJXMYiPX4kReihu5iTvjTpJZPMilOJNXYiIQd8aNPMSDXIkzeSlmQtwZN7KJB7kSZ/JSLIS4M0A2sScncUVeioUQdwbIKp7IUVySV2IlxJ1xI4t4IkdxRV6LhRB3xkTu4kBINnFJXouVEHfGQogjIXmIC/JGrIS4M96RkE1ckSfJTmwk7owfyCauyLNANrGRuDN+IKs4M3kWIKvYCHFnvCerOJOjuJFVPEjcGW/JIs7kJO5kFWfGO7KKC3IQE1nEBeMdWcQFOYqJLOKC8ZbM4oo8i5ms4sz4iUBck71YyCrOjK/IQ6xkFWfGd2QVK9nEmfEtmcVKNnFmfEsmsZFNnBnfk5vYyCbOjAGEeJBNnBnDySYuGKPJQ1wwBpO9ODMGk704MwaTvTgzxpK9uGCMJpu4Yowns7hk/Dnjzxl/zvhz/wHX8lUQ8fABEgAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABaCAAAAACuizbEAAACF0lEQVRoBb3BC3biMADAQOn+h9byKXRLm2DHecwYH2d8nPFxxmnMGGCcQf4Tf5CLuDLWyS/xZMhN3BiL5G9xJU/xxVgio+LBOEzGxTfjGBkXPxhHyLh4YcyTCfHKmCbj4jdjlsyIO4G4MebInLiSu7gwpsikAHmIC2OGzEr+F2CMk3WBMUzWBRiD5AwBxiA5RWCMkVMEGEPkFHFhDJEzxJUxQk4RV8YIOUPcGAPkFHFjvCfniBvjPTlH3BhvyYLkJow74x05LpCbeDLekOMChIR4MvbJcXEhkPHNeEMOiwshJJ6MfXJYXEn8ZOySw+JO4gdjlxwVD8YPxh45KjYZe+SY2GHskUNij7FHjohdxg45IvYZO+SAeMPYJgfEO8Y2mZNkvGNskylhDDC2yYzAGGBskikxyNgkM2KUsUVmxDBjk4yLccYWGRcTjC0yLGYYW2RUTDG2yKCYY2yTETHJ2GTIWzHL+IMkY2Ka8UomxDzjB5kRRxhPMiuOMO5kWhxjXMm8OMi4kmlxlAEyLY4zQKbFcQbIpFhhgMyJJQbIjuSXWGEg2wLkRSwxQP4UX4RAnmKFcWHIi3ghT7HCeDB5iN/kKVYY/zPJ+Is8xQpjlDzFCmOUfIsFxih5ihXGKHmIJcYoeYglxiD5EouMQfIlFhlj5CaMRcYQuYt1xhC5CIllxhBJwlhnjDGJUxhjjLMYH2d8nPFx/wCZfNkBhcdNWAAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA3CAAAAAAa4SpAAAABWElEQVRYCb3BAWKiQADAwOT/j86xaqnaa0WBnTGmM6YzpjOOYGZmZmZmZsYz4yPygbgw3iSfi8HYTPaKwdhGDhCDsYkcIQZjEwNkpxiMN8g+MRjbyV6xMLaTnWIw3iD7xGC8QfaJwdjKkH1iMH4QMjMzMzPZLwbjBzlLDMYzOU0MxhM5TwzGEzlPDMZKzhaDsZKzxWCs5GwxGCs5WwzGlcwQC2OQOWJhLGSSWBgLmSMGA2SWWBjILDEYYDJHLIxB5oiFsZBZAoyFzBJggEwTYIBME2AsZJYAA2SewACZJRYGyDbJL5LX4soA2SIuzMwMYyX3AuRR3BgLeSn+JndikAdxYyzklXhJVjHIg7gxBnkhXpKbuJE78cW4kDvJs9jCJFbyLVbGhXwLeRKfkZv4ZnyRqwB5FB+SIe4ZK0niQu7EoYz/k1Ucy/iFXMXRjD+YcThjOmM6Y7p/c3mdARXOc4QAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA4CAAAAAAJa4PsAAABO0lEQVRYCb3BAXLiMADAQOn/j9aFUNrCAQlj17vGesZ6xnrGesZ6xguSfIuJjKdMHsQsxhPyTExiPJCXYg7jnrwRUxh35K2YwfhFDsQMxg85EjMY3+RYTGDcyBkxzvgip8Q440rOiXHGlZwT44ydnBTjjJ2cFcOMnXwgxhg7+URgci8w45hxIZ8JeS0zHhg3xkb+SuwEMq6MjawRO2Mji8SFcSGLxMbYyDIBxoWsEmBsZJXYGBeyQsaFcSF/IgOBhLgxNjJdvGZcyDRxyNjJpxISMskkiTOMnZwVQowwdnJSjDOu5JwYZ1zJOTHOuJJzYpzxRU6JccYXOSXGGTdyRowzvsmxmMD4IYdiAuMXORAzGL/JezGDcUfeiSmMB/JSzGH8T56KSYxnTB7FLMZzci/mMdYz1jPWM9b7B9JugAEuFMLuAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAIIAAACCCAAAAACvaE8hAAAC80lEQVR4AcXBAY7bOADAQPL/j+a1V2DjbGRHcmxkxvg249uMbzO+zRiTf+JuxpA8xK2MAXkSdzJeyS9xI+OFvIj7GL/JQNzGeCZDcRvjieyIuxgbsivuYmzIvriJ8UOOxE2MH3IkbmI8SJjJSNzDGJCBuIcxJK/iFsaQvIpbGGPyIm5h7JDf4hbGDnkRdzB2yIu4g7FDXsQdjB3yIu5g7JAXcQdjh7yIOxg75FXcwBiTgbiBMSYDcQNjTEbiesaYDMQNjDEZiesZQzIU1zPGZCguZ4zJUFzOGJIdcTVjSHbE1Ywh2RFXM4ZkT1zMGJFdcTFjRPbFtYwR2RfXMkbkQFzKGJEDcSljRI7ElYwBORRXMkbkUFzIGJBjcSFjRI7FdYwBeScuYwzIW3EVY0TeiosYAzIhrmEMyIy4hDEgM+ISxoBMiSsYIzIlLmAMyKT4nDEgs+JjxojMik8ZAzItPmWMyLT4kDEgC+IzxogsiI8YI7IiPmGMy

results:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABzCAAAAACOKGI6AAADdklEQVRoBb3BCWLiSADAQOn/j9aCL4xtCFl6UmX8OePPGX/O+BXja8ZvCMSXjE/JIr5jfEL24ivGB+RJfMX4mRzEN4wfyYX434yfyAvx/xg/kDcyfs14T34Qv2W8Jz+K3zHekw/EbxhvyUfiF4y35EPxMeMd+Vx8yHhHfiE+Y7wjvxCfMd6QX4mPGG/I74RhvGe8If9TvGG8Jv9fvGS8Jt+IF4xX5FtxyXhBBogLxjUZIs6MazJGnBiXZJQ4Mi7JMHFgXJGB4plxRUaKJ8YFGSx2jAsyWOwYZzJcPBhnMl5sjBP5F2JlHMk/ESvjSP6NWBgH8o/Ewngm/0zMjCfyD8XE2JMXku/FxNiRV0K+FhNjIy+FDBB3xkIWIUchA8SdMZNFIEfJCHFnTGQSN3IUMkTcGCCTmMlRyBBxY4DcxZ1cSAYJMEAgJnIhZJAAA4yVnIWMEmCcyF7cyCABxpHsxY0MExhH8hATGSXAOJJNzGSUAONAVrGQcQLjQFaxkGECjANZxUwGCowDWcRCxgkwnskqFjJQYDyTRaxknADjmSxiJeMEGE9kFSsZJm6MZzKLlYwTN8YTWcVChok744nM4kEGiYnxRGaxkVFiYuzJLB5klJgYezKLBxklJsaezGJHBomJsSOz2JFRYmLsyCz25HsJxMTYkVnsyUvJR0IgJsaOTOJB3ogb+VHITUyMB5nFg7wWM3kvQCAmxkZm8SCvxI7JS3EjEBNjJYvYyAtxIC/FjUBMjJXMYiPX4kReihu5iTvjTpJZPMilOJNXYiIQd8aNPMSDXIkzeSlmQtwZN7KJB7kSZ/JSLIS4M0A2sScncUVeioUQdwbIKp7IUVySV2IlxJ1xI4t4IkdxRV6LhRB3xkTu4kBINnFJXouVEHfGQogjIXmIC/JGrIS4M96RkE1ckSfJTmwk7owfyCauyLNANrGRuDN+IKs4M3kWIKvYCHFnvCerOJOjuJFVPEjcGW/JIs7kJO5kFWfGO7KKC3IQE1nEBeMdWcQFOYqJLOKC8ZbM4oo8i5ms4sz4iUBck71YyCrOjK/IQ6xkFWfGd2QVK9nEmfEtmcVKNnFmfEsmsZFNnBnfk5vYyCbOjAGEeJBNnBnDySYuGKPJQ1wwBpO9ODMGk704MwaTvTgzxpK9uGCMJpu4Yowns7hk/Dnjzxl/zvhz/wHX8lUQ8fABEgAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABaCAAAAACuizbEAAACF0lEQVRoBb3BC3biMADAQOn+h9byKXRLm2DHecwYH2d8nPFxxmnMGGCcQf4Tf5CLuDLWyS/xZMhN3BiL5G9xJU/xxVgio+LBOEzGxTfjGBkXPxhHyLh4YcyTCfHKmCbj4jdjlsyIO4G4MebInLiSu7gwpsikAHmIC2OGzEr+F2CMk3WBMUzWBRiD5AwBxiA5RWCMkVMEGEPkFHFhDJEzxJUxQk4RV8YIOUPcGAPkFHFjvCfniBvjPTlH3BhvyYLkJow74x05LpCbeDLekOMChIR4MvbJcXEhkPHNeEMOiwshJJ6MfXJYXEn8ZOySw+JO4gdjlxwVD8YPxh45KjYZe+SY2GHskUNij7FHjohdxg45IvYZO+SAeMPYJgfEO8Y2mZNkvGNskylhDDC2yYzAGGBskikxyNgkM2KUsUVmxDBjk4yLccYWGRcTjC0yLGYYW2RUTDG2yKCYY2yTETHJ2GTIWzHL+IMkY2Ka8UomxDzjB5kRRxhPMiuOMO5kWhxjXMm8OMi4kmlxlAEyLY4zQKbFcQbIpFhhgMyJJQbIjuSXWGEg2wLkRSwxQP4UX4RAnmKFcWHIi3ghT7HCeDB5iN/kKVYY/zPJ+Is8xQpjlDzFCmOUfIsFxih5ihXGKHmIJcYoeYglxiD5EouMQfIlFhlj5CaMRcYQuYt1xhC5CIllxhBJwlhnjDGJUxhjjLMYH2d8nPFx/wCZfNkBhcdNWAAAAABJRU5ErkJggg=="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA3CAAAAAAa4SpAAAABWElEQVRYCb3BAWKiQADAwOT/j86xaqnaa0WBnTGmM6YzpjOOYGZmZmZmZsYz4yPygbgw3iSfi8HYTPaKwdhGDhCDsYkcIQZjEwNkpxiMN8g+MRjbyV6xMLaTnWIw3iD7xGC8QfaJwdjKkH1iMH4QMjMzMzPZLwbjBzlLDMYzOU0MxhM5TwzGEzlPDMZKzhaDsZKzxWCs5GwxGCs5WwzGlcwQC2OQOWJhLGSSWBgLmSMGA2SWWBjILDEYYDJHLIxB5oiFsZBZAoyFzBJggEwTYIBME2AsZJYAA2SewACZJRYGyDbJL5LX4soA2SIuzMwMYyX3AuRR3BgLeSn+JndikAdxYyzklXhJVjHIg7gxBnkhXpKbuJE78cW4kDvJs9jCJFbyLVbGhXwLeRKfkZv4ZnyRqwB5FB+SIe4ZK0niQu7EoYz/k1Ucy/iFXMXRjD+YcThjOmM6Y7p/c3mdARXOc4QAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA4CAAAAAAJa4PsAAABO0lEQVRYCb3BAXLiMADAQOn/j9aFUNrCAQlj17vGesZ6xnrGesZ6xguSfIuJjKdMHsQsxhPyTExiPJCXYg7jnrwRUxh35K2YwfhFDsQMxg85EjMY3+RYTGDcyBkxzvgip8Q440rOiXHGlZwT44ydnBTjjJ2cFcOMnXwgxhg7+URgci8w45hxIZ8JeS0zHhg3xkb+SuwEMq6MjawRO2Mji8SFcSGLxMbYyDIBxoWsEmBsZJXYGBeyQsaFcSF/IgOBhLgxNjJdvGZcyDRxyNjJpxISMskkiTOMnZwVQowwdnJSjDOu5JwYZ1zJOTHOuJJzYpzxRU6JccYXOSXGGTdyRowzvsmxmMD4IYdiAuMXORAzGL/JezGDcUfeiSmMB/JSzGH8T56KSYxnTB7FLMZzci/mMdYz1jPWM9b7B9JugAEuFMLuAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAIIAAACCCAAAAACvaE8hAAAC80lEQVR4AcXBAY7bOADAQPL/j+a1V2DjbGRHcmxkxvg249uMbzO+zRiTf+JuxpA8xK2MAXkSdzJeyS9xI+OFvIj7GL/JQNzGeCZDcRvjieyIuxgbsivuYmzIvriJ8UOOxE2MH3IkbmI8SJjJSNzDGJCBuIcxJK/iFsaQvIpbGGPyIm5h7JDf4hbGDnkRdzB2yIu4g7FDXsQdjB3yIu5g7JAXcQdjh7yIOxg75FXcwBiTgbiBMSYDcQNjTEbiesaYDMQNjDEZiesZQzIU1zPGZCguZ4zJUFzOGJIdcTVjSHbE1Ywh2RFXM4ZkT1zMGJFdcTFjRPbFtYwR2RfXMkbkQFzKGJEDcSljRI7ElYwBORRXMkbkUFzIGJBjcSFjRI7FdYwBeScuYwzIW3EVY0TeiosYAzIhrmEMyIy4hDEgM+ISxoBMiSsYIzIlLmAMyKT4nDEgs+JjxojMik8ZAzItPmWMyLT4kDEgC+IzxogsiI8YI7IiPmGMy